# Baseado nos testes anteriores com esta base e com a base ChestXRay:

* Extrair features para numpy com imageaugmented bem "suave" (teste 01b3)
   * Rodar com maxpool e com avgpool para poder comparar
* Rodar keras_tuner e comparar resultados com melhor resultado da rede simples
* Criar matriz e testar busca interativamente com as features da imagenet
* Reduzir com PCA ou T-SNE e testar se prejudica busca
* Testar clusterização (para colocar busca em produção mesmo com dezenas de milhões de imagens)

### Os resultados da rede simples treinada do zero foram bons, mas caso os resultados com rede pré treinada na imagenet sejam bons, pode ser um método universal para vários classificadores, buscas e análises pré extrair features via rede pré treinada, salvando em Banco de Dados.

In [5]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
    ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import RMSprop

from utils import get_imagens_erro, report, plot_errors
import time


caminho_bases = os.path.join('..', 'bases', 'vazios')
caminho_train = os.path.join(caminho_bases, 'train')
caminho_test = os.path.join(caminho_bases, 'test')

MODEL_DIR = '../models_featureextraction'
LOG_DIR = '../logs/02c2/'
SIZE = (224, 224)
destino_bases = os.path.join('..', 'bases', 'vazios_featureextraction')
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
if not os.path.exists(caminho_bases):
    os.mkdir(caminho_bases)


# Callbacks

In [106]:
tensorboard_logs = TensorBoard(log_dir=LOG_DIR, histogram_freq=1,
                               write_graph=False, write_images=False,
                               update_freq='epoch')
mcp_save = ModelCheckpoint(os.path.join(MODEL_DIR, 
                                        'Transfermodelweights02c2.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2,
                              verbose=1, mode='min')


# Model for feature extraction

In [7]:
# create the base pre-trained model
base_model = DenseNet121(weights='imagenet',
                         input_shape=(*SIZE, 3), 
                         include_top=False,
                         pooling='max')

# DataGenerators

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.05,
    brightness_range=[0.95, 1.05],
    fill_mode='nearest',
    horizontal_flip=True,
)
train_generator = train_datagen.flow_from_directory(
    caminho_train,
    target_size=SIZE,
    batch_size=64,
    class_mode='binary',
    shuffle=False
)

Found 20845 images belonging to 2 classes.


In [9]:
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.05,
    brightness_range=[0.95, 1.05],
    fill_mode='nearest',
    horizontal_flip=True,
)

validation_generator = validation_datagen.flow_from_directory(
    caminho_test,
    target_size=SIZE,
    batch_size=64,
    class_mode='binary',
    shuffle=False
)

Found 2317 images belonging to 2 classes.


# Extract features to np

In [10]:
FEATURES_DIR = destino_bases

In [4]:
# Se tiver sido gerado anteriormente, basta ler do disco
with open(os.path.join(FEATURES_DIR, 'features_train.np'), 'rb') as features_in:
    features_train = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_train.np'), 'rb') as y_in:
    y_train = np.load(y_in)
with open(os.path.join(FEATURES_DIR, 'features_test.np'), 'rb') as features_in:
    features_test = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_test.np'), 'rb') as y_in:
    y_test = np.load(y_in)

In [35]:
def extract_features(generator, model):
    generator.reset()
    n_images = len(generator.filenames)
    batch_size = generator.batch_size
    m = batch_size *  (n_images // batch_size)  # Arredondar para não ficar espaço vazio
    n = model.output.shape[1]
    features = np.zeros((m, n), np.float32)
    y_ = np.zeros((m, 1), np.float32)
    print(n_images, m, batch_size, m // batch_size)
    for ind in range(m // batch_size):
        batch, y_batch = next(generator)
        features_batch = base_model.predict(batch)
        features[ind * batch_size: (ind * batch_size) + batch_size, :] = features_batch
        y_[ind * batch_size: (ind * batch_size) + batch_size, 0] = y_batch
    print('last batch setted elements %s:%s' % ((ind * batch_size), ((ind * batch_size) + batch_size)))
    return features, y_

In [37]:
features_train, y_train = extract_features(train_generator, base_model)

20845 20800 64 325
last batch setted elements 20736:20800


In [38]:
print(features_train.shape)
print(y_train.shape)

(20800, 1024)
(20800, 1)


In [36]:
features_test, y_test = extract_features(validation_generator, base_model)

2317 2304 64 36
last batch setted elements 2240:2304


In [39]:
print(features_test.shape)
print(y_test.shape)

(2304, 1024)
(2304, 1)


In [40]:
with open(os.path.join(FEATURES_DIR, 'features_train.np'), 'bw') as features_out:
    np.save(features_out, features_train)
with open(os.path.join(FEATURES_DIR, 'y_train.np'), 'bw') as y_out:
    np.save(y_out, y_train)

In [41]:
with open(os.path.join(FEATURES_DIR, 'features_test.np'), 'bw') as features_out:
    np.save(features_out, features_test)
with open(os.path.join(FEATURES_DIR, 'y_test.np'), 'bw') as y_out:
    np.save(y_out, y_test)

# Model for feature extraction (AvgPooling)

In [42]:
# create the base pre-trained model
base_model_avg = DenseNet121(weights='imagenet',
                         input_shape=(*SIZE, 3), 
                         include_top=False,
                         pooling='avg')

# Extract features to np (AvgPooling)

In [ ]:
# Se tiver sido gerado anteriormente, basta ler do disco
with open(os.path.join(FEATURES_DIR, 'features_train_avg.np'), 'rb') as features_in:
    features_train_avg = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_train_avg.np'), 'rb') as y_in:
    y_train_avg = np.load(y_in)
with open(os.path.join(FEATURES_DIR, 'features_test_avg.np'), 'rb') as features_in:
    features_test_avg = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_test_avg.np'), 'rb') as y_in:
    y_test_avg = np.load(y_in)

In [43]:
features_train_avg, y_train_avg = extract_features(train_generator, base_model_avg)

20845 20800 64 325
last batch setted elements 20736:20800


In [44]:
print(features_train_avg.shape)
print(y_train_avg.shape)

(20800, 1024)
(20800, 1)


In [45]:
features_test_avg, y_test_avg = extract_features(validation_generator, base_model_avg)

2317 2304 64 36
last batch setted elements 2240:2304


In [46]:
print(features_test_avg.shape)
print(y_test_avg.shape)

(2304, 1024)
(2304, 1)


In [47]:
with open(os.path.join(FEATURES_DIR, 'features_train_avg.np'), 'bw') as features_out:
    np.save(features_out, features_train_avg)
with open(os.path.join(FEATURES_DIR, 'y_train_avg.np'), 'bw') as y_out:
    np.save(y_out, y_train_avg)

In [48]:
with open(os.path.join(FEATURES_DIR, 'features_test_avg.np'), 'bw') as features_out:
    np.save(features_out, features_test_avg)
with open(os.path.join(FEATURES_DIR, 'y_test_avg.np'), 'bw') as y_out:
    np.save(y_out, y_test_avg)

# Liberar memória

In [49]:
del base_model
del base_model_avg

# Model RandomSearch KerasTuner

In [86]:
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=128,
                           activation='relu',
                           input_shape=(1024,)))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=128,
                           activation='relu'))
    model.add(tf.keras.layers.Dropout(hp.Choice('lrate',
                                      values=[0.2, 0.4])))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[0.0001, 0.00004, 0.00001])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [79]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=2,
    directory='kerastunerlogs',
    project_name='Vazios')

In [80]:
tuner.search_space_summary()

# Tuner - Teste com MaxPool

In [81]:
tuner.search(x=features_train,
             y=y_train,
             epochs=10,
             validation_data=(features_test, y_test),
            )

W0710 12:10:39.228868 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.157958). Check your callbacks.
W0710 12:10:39.291945 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.104327). Check your callbacks.


Name,Best model,Current model
accuracy,0.9501,0.9501
val_loss,0.1445,0.1445
loss,0.1635,0.1635
val_accuracy,0.954,0.954


W0710 12:11:46.608629 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.106839). Check your callbacks.
W0710 12:11:46.638159 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.106839). Check your callbacks.
W0710 12:11:46.678763 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.106839). Check your callbacks.
W0710 12:11:46.700974 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.106839). Check your callbacks.


Name,Best model,Current model
accuracy,0.9549,0.9549
val_loss,0.1402,0.1402
loss,0.1461,0.1461
val_accuracy,0.9557,0.9557


Oracle triggered exit


In [82]:
tuner.results_summary()

# Tuner - Teste com AvgPool

In [83]:
tuner.search(x=features_train_avg,
             y=y_train_avg,
             epochs=10,
             validation_data=(features_test_avg, y_test_avg),
            )

W0710 12:13:16.168517 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.200158). Check your callbacks.
W0710 12:13:16.202365 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.109022). Check your callbacks.


W0710 12:13:35.043132 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.109354). Check your callbacks.
W0710 12:13:35.259618 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.112996). Check your callbacks.
W0710 12:13:35.408587 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.112996). Check your callbacks.
W0710 12:13:35.523991 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.112996). Check your callbacks.
W0710 12:13:35.559537 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107065). Check your callbacks.
W0710 12:13:35.689697 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107065). Check your callbacks.
W0710 12:13:35.719903 139904768079616 callbacks.py:241] Method (on_tra

W0710 12:13:49.451974 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.119044). Check your callbacks.
W0710 12:13:49.608206 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.138525). Check your callbacks.
W0710 12:13:49.729732 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.138525). Check your callbacks.
W0710 12:13:49.766310 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.138525). Check your callbacks.
W0710 12:13:49.778784 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.127418). Check your callbacks.
W0710 12:13:49.791177 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107937). Check your callbacks.


W0710 12:14:32.717909 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.112083). Check your callbacks.
W0710 12:14:32.837350 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.112083). Check your callbacks.
W0710 12:14:32.963488 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.114096). Check your callbacks.
W0710 12:14:33.090873 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.114168). Check your callbacks.
W0710 12:14:33.219648 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.115007). Check your callbacks.
W0710 12:14:33.356442 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.115007). Check your callbacks.
W0710 12:14:33.384125 139904768079616 callbacks.py:241] Method (on_tra

W0710 12:14:55.397021 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.303431). Check your callbacks.
W0710 12:14:55.409013 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.155027). Check your callbacks.
W0710 12:15:03.769032 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.135326). Check your callbacks.
W0710 12:15:03.797059 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.135326). Check your callbacks.
W0710 12:15:03.810311 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.135326). Check your callbacks.
W0710 12:15:03.820936 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.135326). Check your callbacks.
W0710 12:15:03.832063 139904768079616 callbacks.py:241] Method (on_tra

Name,Best model,Current model
accuracy,0.9549,0.933
val_loss,0.1402,0.1718
loss,0.1461,0.2085
val_accuracy,0.9557,0.9481


Oracle triggered exit


In [84]:
tuner.results_summary()

# Final Training and validation

In [114]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=128,
                       activation='relu',
                       input_shape=(1024,)))
model.add(tf.keras.layers.Dropout(0.33))
model.add(tf.keras.layers.Dense(units=128,
                       activation='relu',
                       input_shape=(1024,)))
model.add(tf.keras.layers.Dropout(0.33))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00004),
    loss='binary_crossentropy',
    metrics=['accuracy'])


In [115]:
epoch = 0

In [116]:
val_loss = 0
model.load_weights(
    os.path.join(MODEL_DIR,
                 'Transfermodelweights02c.{:02d}-{:.2f}.hdf5'.format(epoch, val_loss)
                ))

OSError: Unable to open file (unable to open file: name = '../models_featureextraction/Transfermodelweights02c.00-0.00.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [117]:
history = model.fit(x=features_train,
                    y=y_train,
                    batch_size=128,
                    initial_epoch=epoch,
                    epochs=200,
                    verbose=2,
                    validation_data=(features_test, y_test),
                    callbacks=[early_stop, mcp_save, reduce_lr, tensorboard_logs],
                   )

Train on 20800 samples, validate on 2304 samples
Epoch 1/200
20800/20800 - 1s - loss: 0.6539 - accuracy: 0.7139 - val_loss: 0.2625 - val_accuracy: 0.9167
Epoch 2/200
20800/20800 - 0s - loss: 0.3383 - accuracy: 0.8655 - val_loss: 0.2207 - val_accuracy: 0.9301
Epoch 3/200
20800/20800 - 0s - loss: 0.2862 - accuracy: 0.8927 - val_loss: 0.1990 - val_accuracy: 0.9414
Epoch 4/200
20800/20800 - 0s - loss: 0.2474 - accuracy: 0.9104 - val_loss: 0.1896 - val_accuracy: 0.9423
Epoch 5/200
20800/20800 - 0s - loss: 0.2360 - accuracy: 0.9200 - val_loss: 0.1801 - val_accuracy: 0.9462
Epoch 6/200
20800/20800 - 0s - loss: 0.2196 - accuracy: 0.9280 - val_loss: 0.1745 - val_accuracy: 0.9475
Epoch 7/200
20800/20800 - 0s - loss: 0.2101 - accuracy: 0.9317 - val_loss: 0.1720 - val_accuracy: 0.9484
Epoch 8/200
20800/20800 - 0s - loss: 0.2040 - accuracy: 0.9345 - val_loss: 0.1642 - val_accuracy: 0.9479
Epoch 9/200
20800/20800 - 0s - loss: 0.1961 - accuracy: 0.9377 - val_loss: 0.1602 - val_accuracy: 0.9488
Epoch 

In [118]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00004),
    loss='binary_crossentropy',
    metrics=['accuracy'])



In [119]:
history = model.fit(x=features_train_avg,
                    y=y_train_avg,
                    batch_size=128,
                    initial_epoch=epoch,
                    epochs=200,
                    verbose=2,
                    validation_data=(features_test_avg, y_test_avg),
                    callbacks=[early_stop, mcp_save, reduce_lr, tensorboard_logs],
                   )

Train on 20800 samples, validate on 2304 samples
Epoch 1/200
20800/20800 - 1s - loss: 0.1601 - accuracy: 0.9516 - val_loss: 0.1437 - val_accuracy: 0.9549
Epoch 2/200
20800/20800 - 0s - loss: 0.1597 - accuracy: 0.9512 - val_loss: 0.1435 - val_accuracy: 0.9557
Epoch 3/200
20800/20800 - 0s - loss: 0.1555 - accuracy: 0.9520 - val_loss: 0.1432 - val_accuracy: 0.9557
Epoch 4/200
20800/20800 - 0s - loss: 0.1537 - accuracy: 0.9529 - val_loss: 0.1420 - val_accuracy: 0.9562
Epoch 5/200
20800/20800 - 0s - loss: 0.1550 - accuracy: 0.9520 - val_loss: 0.1406 - val_accuracy: 0.9562
Epoch 6/200
20800/20800 - 0s - loss: 0.1527 - accuracy: 0.9537 - val_loss: 0.1411 - val_accuracy: 0.9557
Epoch 7/200

Epoch 00007: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
20800/20800 - 0s - loss: 0.1515 - accuracy: 0.9536 - val_loss: 0.1411 - val_accuracy: 0.9562
Epoch 8/200
20800/20800 - 0s - loss: 0.1503 - accuracy: 0.9528 - val_loss: 0.1399 - val_accuracy: 0.9566
Epoch 9/200
20800/20800 - 0s 